In [2]:
from dataclasses import dataclass, field
from enum import Enum
from typing import List, Union, Optional


class LogicOperator(Enum):
    """
    The set of accepted logic operators,
    the first value is there priority, like pemdas
    the second value is there operand count binary, unary, etc.
    """
    OR = 1,2
    AND = 2,2
    NOT = 3,1
    LPAREN = 4,0
    RPAREN = 5,0

class LogicTerminal(Enum):
    U = -1 # Unknown
    F = 0 # False
    T = 1 # True

class LogicExpression:
    def  __init__(self,expression: List[Union[LogicOperator, LogicTerminal, str]] = None):
        self.variables = {}
        self.expression = []
        self.iter_count = 0
        if expression:
            self.set_expression(expression)

    def __str__(self):
        parts = []
        for value in self.expression:
            if isinstance(value, str): parts.append(value)
            else: parts.append(value.name)
        return " ".join(parts)

    def __next__(self):
        if self.iter_count < len(self.expression):
            value = self.expression[self.iter_count]
            self.iter_count += 1
            return value
        else:
            raise StopIteration

    def __iter__(self):
        self.iter_count = 0
        return self

    def peek(self)-> Union[LogicOperator, LogicTerminal, str, None]:
        if self.iter_count < len(self.expression):
            return self.expression[self.iter_count]
        else:
            return None

    def set_expression(self, expression: List[Union[LogicOperator, LogicTerminal, str]]):
        """
        :param expression: Takes a list as an input, the list is an ordered sequence of 3 possible types:
        --Logic Operators using the LogicOperator Enumerator
        --Logic Terminals using the LogicTerminal enumerator
        --Strings, these can be any value and represent variables within the logic.
        This value is set and stored.
        """
        for value in expression:
            if isinstance(value, str):
                self.variables[value] = LogicTerminal.U

        self.expression = expression

    def set_variable(self, variable, value) -> bool:
        if variable not in self.variables:
            return False
        self.variables[variable] = value
        return True

    def get_variable(self, variable):
        return self.variables[variable]

@dataclass
class ParseNode:
    nodeType: Union['ParseNode', 'LogicOperator', 'LogicTerminal', str, None] = None
    children: List['ParseNode'] = field(default_factory=list)

class ExpressionParser:
    def __init__(self):
        self.expression = None   # will hold the LogicExpression during parsing

    def parse_expression(self, expression: LogicExpression) -> ParseNode:
        self.expression = expression
        self.expression.iter_count = 0  # reset iterator
        node = self._parse_or()
        if self.expression.peek() is not None:
            raise ValueError(f"Expression not empty after parse: {self.expression.peek()}")
        return node

    # Rules in order
    def _parse_or(self) -> ParseNode:
        node = self._parse_and()
        while self.expression.peek() == LogicOperator.OR:
            next(self.expression)  # consume OR
            rhs = self._parse_and()
            node = ParseNode(nodeType=LogicOperator.OR, children=[node, rhs])
        return node

    def _parse_and(self) -> ParseNode:
        node = self._parse_not()
        while self.expression.peek() == LogicOperator.AND:
            next(self.expression)  # consume AND
            rhs = self._parse_not()
            node = ParseNode(nodeType=LogicOperator.AND, children=[node, rhs])
        return node

    def _parse_not(self) -> ParseNode:
        if self.expression.peek() == LogicOperator.NOT:
            next(self.expression)  # consume NOT
            child = self._parse_not()
            return ParseNode(nodeType=LogicOperator.NOT, children=[child])
        return self._parse_atom()

    def _parse_atom(self) -> ParseNode:
        atom = next(self.expression)  # consume one
        if isinstance(atom, str):
            return ParseNode(nodeType=atom)
        if isinstance(atom, LogicTerminal):
            return ParseNode(nodeType=atom)
        if atom == LogicOperator.LPAREN:
            node = self._parse_or()
            if self.expression.peek() != LogicOperator.RPAREN:
                raise ValueError("Expected ')'")
            next(self.expression)  # consume RPAREN
            return node
        raise ValueError(f"Unexpected value: {atom}")

    @staticmethod
    def pretty_print(node: ParseNode, indent: str = "", is_last: bool = True):
        # Decide connector symbols
        branch = "└── " if is_last else "├── "
        next_indent = indent + ("    " if is_last else "│   ")

        # Format node label
        if isinstance(node.nodeType, LogicOperator):
            label = str(node.nodeType.name)
        elif isinstance(node.nodeType, LogicTerminal):
            label = str(node.nodeType.name)
        else:
            label = str(node.nodeType)

        # Print this node
        print(indent + branch + label)

        # Recurse on children
        for i, child in enumerate(node.children):
            ExpressionParser.pretty_print(child, next_indent, i == len(node.children) - 1)




a = ExpressionParser()
# x AND (NOT y) OR T
expression = LogicExpression([
    "x",                          # variable
    LogicOperator.AND,            # AND
    LogicOperator.LPAREN,         # (
        LogicOperator.NOT,        # NOT
        "y",                      # variable
    LogicOperator.RPAREN,         # )
    LogicOperator.OR,             # OR
    LogicTerminal.T               # constant True
])

print(expression)
root = a.parse_expression(expression)
a.pretty_print(root)





x AND LPAREN NOT y RPAREN OR T
└── OR
    ├── AND
    │   ├── x
    │   └── NOT
    │       └── y
    └── T
